In [34]:
import pandas as pd 
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_1.csv')
df['총대출액'] = df['금액1'] + df['금액2']
df_group = df.groupby(['year', 'gender', '지역코드']).총대출액.sum().reset_index(drop=False)
df_piv = pd.pivot_table(df_group, index=['year', '지역코드'], columns = ['gender'], values = '총대출액', fill_value = 0)
df_piv['abs'] = abs(df_piv[0] - df_piv[1])
print(df_piv['abs'].idxmax()[1])

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_2.csv')
#print(df.head(2))
df_totl = df[df['구분'] == '발생건수']
df_totl = df_totl.drop('구분', axis=1)
df_totl = df_totl.set_index('연도')
df_true = df[df['구분'] == '검거건수']
df_true = df_true.drop('구분', axis=1)
df_true = df_true.set_index('연도')
df_ratio = df_true / df_totl
print('Q2:', int(df_true[df_ratio >= 1].sum().sum()))

import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_3.csv')
#print(df.head(2))
df['평균만족도'] = df['평균만족도'].fillna(df['평균만족도'].mean())
df['근속연수'] = df['근속연수'].fillna(np.floor(df.groupby(['부서', '등급'])['근속연수'].transform('mean')))
print('Q3-1:', df[(df['부서'] == 'HR') & (df['등급'] == 'A')]['근속연수'].mean())
print('Q3-2:', df[(df['부서'] == 'Sales') & (df['등급'] == 'B')]['교육참가횟수'].mean())
print('Q3-3:', df[(df['부서'] == 'HR') & (df['등급'] == 'A')]['근속연수'].mean() + 
      df[(df['부서'] == 'Sales') & (df['등급'] == 'B')]['교육참가횟수'].mean())

143
Q2: 36977
Q3-1: 17.625
Q3-2: 7.6
Q3-3: 25.225


In [99]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_test.csv')

y_train = train['라벨']
key = test['ID']
x_train = train.drop(['ID', '라벨'], axis=1)
y_test = test['라벨']
x_test = test.drop(['ID', '라벨'], axis=1)

merge = pd.concat([x_train, x_test], axis=0)
obj = ['지역', '등급']
merge = pd.get_dummies(merge, columns=obj)
merge['연도'] = merge['연도'] - 2000
x_train = merge.iloc[:len(x_train), :]
x_test = merge.iloc[len(x_train):, :]

from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators = 1000,
    max_depth = 8,
    min_samples_leaf = 4,
    min_samples_split = 5,
    random_state = 42,
    class_weight = 'balanced'
)

model.fit(x_train, y_train)
pred = model.predict(x_test)
f1_macro = f1_score(y_test, pred, average='macro')
print('F1 Macro:', f1_macro)
result = pd.DataFrame()
result['ID'] = key
result['pred'] = pred
print(result.head(2))
print(result.isnull().sum())

F1 Macro: 0.34564392377897196
     ID  pred
0  1094     1
1  1341     0
ID      0
pred    0
dtype: int64


In [140]:
import statsmodels.formula.api as smf

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_3_1.csv')

train = df[:140]
test = df[140:]
model = smf.ols('design ~ tenure + f2 + C(f3) + f4 + f5', data=train).fit()
print(model.summary())
print('Q1:', len(model.params[model.pvalues >= 0.05]))

from scipy.stats import pearsonr

pred = model.predict(train)
corr, p_value = pearsonr(pred, train['design'])
print('Q2:', round(corr, 3))
print('Q2:', round(np.sqrt(model.rsquared),3))

from sklearn.metrics import root_mean_squared_error

pred_t = model.predict(test)
rmse = root_mean_squared_error(test['design'], pred_t)
print('Q3:', rmse)

                            OLS Regression Results                            
Dep. Variable:                 design   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     137.4
Date:                Tue, 25 Nov 2025   Prob (F-statistic):           5.91e-51
Time:                        14:17:40   Log-Likelihood:                -421.42
No. Observations:                 140   AIC:                             854.8
Df Residuals:                     134   BIC:                             872.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5393      2.584      0.983      0.3

In [166]:
import statsmodels.formula.api as smf

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_3_2.csv')
print(df.head(2))
model = smf.logit('churn ~ col1 + col2 + C(Phone_Service) + C(Tech_Insurance)', data=df).fit()
print(model.summary())
print('Q1:', round(model.pvalues['col1'], 4))
print('Q2:', round(np.exp(model.params['C(Phone_Service)[T.1]']),4))
pred = model.predict(df)
print('Q3:', (pred>=0.3).sum())

        col1       col2  Phone_Service  Tech_Insurance  churn
0  54.967142  34.630888              0               1      0
1  48.617357  39.547083              0               0      0
Optimization terminated successfully.
         Current function value: 0.640721
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  churn   No. Observations:                  500
Model:                          Logit   Df Residuals:                      495
Method:                           MLE   Df Model:                            4
Date:                Tue, 25 Nov 2025   Pseudo R-squ.:                 0.07562
Time:                        14:32:09   Log-Likelihood:                -320.36
converged:                       True   LL-Null:                       -346.57
Covariance Type:            nonrobust   LLR p-value:                 1.128e-10
                             coef    std err          z      P>|z|      [0.025  